# Start 

<div class="alert alert-block alert-info">
<b>In3dSlicer GUI</b>:
<p>1. Load DICOM folder using "File -> Add DICOM data" panel</p>
<p>2. Save project with any files in the 'results' folder. To do so use "File -> Save Data -> Change directory for the selected files</p>

PLEASE SAVE RESULTS EVERY TIME YOU MOVE TO THE NEXT TASK.

If your results are saved, you can close the notebook. This will kill the managed 3DSlicer window. You can then re-open the notebook, load the libraries and helper functions and continue from the moment you left. EXCEPT, make sure that you are working with the intended orientation of the slices.

</div>

# Load Libraries and Initiate Functions

In [4]:
import numpy as np
import pandas as pd
import os
import slicer
import math
import vtk

from tps import ThinPlateSpline


In [5]:
# helper functions:
def sphereFit(data):
    #   Assemble the A matrix
    spX = np.array(data[:,0])
    spY = np.array(data[:,1])
    spZ = np.array(data[:,2])
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f, rcond = None)

    #   solve for the radius
    t = np.dot(C[0],C[0])+np.dot(C[1],C[1])+np.dot(C[2],C[2])+C[3]
 
    radius = math.sqrt(t[0])
    center = np.array([C[0], C[1], C[2]])

    return radius, center

def plane_normal(p1, p2, p3):
    v1 = p1 - p2
    v2 = p3 - p2
    v3 = np.cross(v1, v2)
    return v3/np.linalg.norm(v3)

def setSlicePoseFromSliceNormalAndPosition(sliceNode, sliceNormal, slicePosition, defaultViewUpDirection=None, backupViewRightDirection=None):
    
    ## Fix up input directions
    if defaultViewUpDirection is None:
        defaultViewUpDirection = [0,0,1]
    if backupViewRightDirection is None:
        backupViewRightDirection = [-1,0,0]
    if sliceNormal[1]>=0:
        sliceNormalStandardized = sliceNormal
    else:
        sliceNormalStandardized = [-sliceNormal[0], -sliceNormal[1], -sliceNormal[2]]

        ## Compute slice axes
    sliceNormalViewUpAngle = vtk.vtkMath.AngleBetweenVectors(sliceNormalStandardized, defaultViewUpDirection)
    angleTooSmallThresholdRad = 0.25 # about 15 degrees
    if sliceNormalViewUpAngle > angleTooSmallThresholdRad and sliceNormalViewUpAngle < vtk.vtkMath.Pi() - angleTooSmallThresholdRad:
        viewUpDirection = defaultViewUpDirection
        sliceAxisY = viewUpDirection
        sliceAxisX = [0, 0, 0]
        vtk.vtkMath.Cross(sliceAxisY, sliceNormalStandardized, sliceAxisX)
    else:
        sliceAxisX = backupViewRightDirection
    
    ## Set slice axes
    sliceNode.SetSliceToRASByNTP(sliceNormalStandardized[0], sliceNormalStandardized[1], sliceNormalStandardized[2],
        sliceAxisX[0], sliceAxisX[1], sliceAxisX[2],
        slicePosition[0], slicePosition[1], slicePosition[2], 0)
    
def update_landmark_position_by_name(markups_node, landmark_name, new_position):
    """
    Update the position of a landmark in a markups node by its name.
    
    """
    # Check if the node is valid
    if not markups_node or not isinstance(markups_node, slicer.vtkMRMLMarkupsNode):
        print("Invalid markups node")
        return False
    
    # Iterate through all landmarks in the markups node
    n_landmarks = markups_node.GetNumberOfControlPoints()
    for i in range(n_landmarks):
        # Get current landmark's name
        current_name = markups_node.GetNthControlPointLabel(i)
        
        # If we found the landmark with the specified name
        if current_name == landmark_name:
            # Update its position
            markups_node.SetNthControlPointPosition(i, *new_position)
            print(f"Updated position of landmark '{landmark_name}' to {new_position}")
            return True
    
    # If we get here, the landmark wasn't found
    print(f"Landmark '{landmark_name}' not found in the markups node")
    return False

In [6]:
# load the template file with landmarks
template_markers_df = pd.read_csv(os.path.join(f'../../', 'templates', 'bone_markers_in_ground.csv'), index_col='name')

# First Set of Landmarks

## Femur Head Centres
Create Point Clouds to Calclulate Femur Head Centers

In [12]:
# create a set of first 21 landmarks with zero postions in slice

In [7]:
first_set_of_landmarks = ['femur_l_center', 'femur_r_center', 'torso_origin_in_pelvis', 
                            'ASIS_l', 'ASIS_r', 'PSIS_l', 'PSIS_r', 'pub_infer_c', 'pub_super_c', 
                            'ilium_l', 'ilium_r', 'fibula_l_lat_malleol_tip','tibia_l_med_malleol_tip', 
                            'fibula_r_lat_malleol_tip', 'tibia_r_med_malleol_tip', 'knee_l_center', 'knee_r_center',
                            'knee_l_lat', 'knee_l_med', 'knee_r_lat', 'knee_r_med']

In [9]:
lms_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
lms_cloud.SetName('lms')
slicer.util.updateMarkupsControlPointsFromArray(lms_cloud, np.zeros((len(first_set_of_landmarks), 3)))
for i, name in enumerate(first_set_of_landmarks):
    lms_cloud.SetNthFiducialLabel(i, name)


In [10]:
# create empty sets of markups for fmur head right and left

In [11]:
femur_head_right = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
femur_head_right.SetName('femur_head_right')
femur_head_left = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
femur_head_left.SetName('femur_head_left')

<div class="alert alert-block alert-info">
<b>In3dSlicer GUI</b>:

<p>For each, <emp>femur_head_right</emp> and <emp>femur_head_left</emp>, collect landmarks on two perpendicular circular outlines of each femur head. Please, use red and green slices for each bone. Make sure to pick slices at the level of the largest femur head radius (approximately near fovea).</p>

<p>Once finished, follow instructions below.</p>
</div>

In [10]:
# left femur
left_fh = slicer.util.getNode('femur_head_left')
left_fh = slicer.util.arrayFromMarkupsControlPoints(left_fh)
left_radius, left_center = sphereFit(left_fh)

In [11]:
# right femur
right_fh = slicer.util.getNode('femur_head_right')
right_fh = slicer.util.arrayFromMarkupsControlPoints(right_fh)
right_radius, right_center = sphereFit(right_fh)

In [12]:
# Import lms and update center points
lms = slicer.util.getNode('lms')
update_landmark_position_by_name(lms, 'femur_l_center', left_center)
update_landmark_position_by_name(lms, 'femur_r_center', right_center)

C:/Users/User/AppData/Local/Temp/xpython_24788/1111638032.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  markups_node.SetNthControlPointPosition(i, *new_position)


Updated position of landmark 'femur_l_center' to [[-73.32891438]
 [-33.92052942]
 [689.13472768]]


Updated position of landmark 'femur_r_center' to [[ 90.69030244]
 [-37.54366773]
 [691.90863225]]


True

## Pelvis

### Landmarks

<div class="alert alert-block alert-info">
    <b>In 3dSlicer GUI</b>, place the following key markers:
    
 1) _torso_origin_in_pelvis_ : defined as a point at the **posterior bottom edge of the L5** on the yellow mid-sagittal slice. This point must later be on the same slice as _pelvis_origin_ and _pelvis_origin_in_ground_ .
   
 3) _ASIS_l_ and _ASIS_r_ : defined as the anteriormost point of the iliac blades that touches skin on the red slice; on the green slice, the same point should look like as small round shadow and on the yellow slice it is anteroinferior point of the iliac blade.
    
 4) _PSIS_l_ and _PSIS_r_ : defined as posterior-most points of iliac blades. They should be positioned posterior to sacrum. On the red slice, the sacrum must still be visible, probably it's uppermost vertebra.
    
 5) _pub_infer_c_ and _pub_super_c_ : defined as the inferior and superior point on the sagittal optline of the pubic symphysis. Please note that the two points should lie along the longest axis of the sympysis.
    
</div>

In [13]:
# re-load lms node
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
asis_l = lms_list[lms.GetControlPointIndexByLabel('ASIS_l')]
asis_r = lms_list[lms.GetControlPointIndexByLabel('ASIS_r')]
print(asis_l, asis_r)

[-105.7529531    19.68432793  760.54521234] [114.46565354  22.54430983 760.54521234]


In [14]:
pelvis_origin = np.mean((asis_l, asis_r), axis = 0)
pelvis_origin_point = lms.AddControlPoint(vtk.vtkVector3d(pelvis_origin))
lms.SetNthControlPointLabel(pelvis_origin_point, 'pelvis_origin_in_ground')

psis_l = lms_list[lms.GetControlPointIndexByLabel('PSIS_l')]
psis_r = lms_list[lms.GetControlPointIndexByLabel('PSIS_r')]
psis_c = np.mean((psis_l, psis_r), axis = 0)

pub_infer_c = lms_list[lms.GetControlPointIndexByLabel('pub_infer_c')]
pub_super_c = lms_list[lms.GetControlPointIndexByLabel('pub_super_c')]

In [15]:
# define new normal for the red slice
red_slice_new_normal_1 = np.cross((pelvis_origin - psis_l), (pelvis_origin - psis_r))
red_slice_new_normal = red_slice_new_normal_1/np.linalg.norm(red_slice_new_normal_1)
red_slice_new_normal

array([-0.052599  ,  0.34265556,  0.93798748])

In [16]:
# reorient red slice along ASIS - PSIS direction
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(
    redSliceNode, red_slice_new_normal, 
    np.mean(((psis_l+psis_r)*0.5, pelvis_origin), 
            axis = 0)) ## red slice now shows PSIS_l, PSIS_r and pelvis_origin points

In [17]:
# reorient green slice
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(
    greenSliceNode, np.array([0,-1,0]), 
    np.mean(((psis_l+psis_r)*0.5, pelvis_origin), 
            axis = 0)) ## green slice now crossects pelvis in the middle

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, collect further key markers:</b>
    
1)  _ilium_l_ and _ilium_r_ : move the red slice up so that it touches the tops of iliac crests. The green slice position marks the centre between psis and pelvis origin markers. The red slice can show elongated shadow(s) of the left or/and right iliac crests that do not quite reach the green line. The red slice then marks the top of the iliac crest on the green slice. Place points at the top of the iliac crest on the green slice.
   </div> 



### Check Orientation Planes:

It should coinside with Rajagopal et al. (2016) pelvis orientation. Once displayed, one can move the red slice to show that is transsects both pubic tubercles.

In [28]:
# reorient yellow slice to be perpendicular ASIS axis:

'''the following two lines are to be used on the first data collection'''
lms = slicer.util.getNode('lms') # deactivate on amendments

'''the following two lines are to be used when testing and making amendments'''
# lms = slicer.util.getNode('orientation')  # activate on amendments
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

asis_r = lms_list[lms.GetControlPointIndexByLabel('ASIS_r')]
asis_l = lms_list[lms.GetControlPointIndexByLabel('ASIS_l')]
pub_super_c = lms_list[lms.GetControlPointIndexByLabel('pub_super_c')]
# pelvis_origin = np.mean((asis_l, asis_r), axis = 0)
# pelvis_origin_point = lms.AddControlPoint(vtk.vtkVector3d(pelvis_origin))
# lms.SetNthControlPointLabel(pelvis_origin_point, 'pelvis_origin')
pelvis_origin = lms_list[lms.GetControlPointIndexByLabel('pelvis_origin')] # deactivate on amendments

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
asis_axis = (asis_r - asis_l)/np.linalg.norm(asis_r - asis_l)
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, asis_axis , pub_super_c)

In [29]:
# reorient green slice to make it anterior as in Rajagopal et al 2016 passing through pub_sup_c:
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
pelvis_origin_to_symph = (pub_super_c - pelvis_origin)/np.linalg.norm(pub_super_c - pelvis_origin)
frontal_norm = np.cross(asis_axis, pelvis_origin_to_symph)
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, frontal_norm, pub_super_c)

In [30]:
# reorient red slice to make it perpendicular to the green above:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
pelvis_origin_to_symph = (pub_super_c - pelvis_origin)/np.linalg.norm(pub_super_c - pelvis_origin)
setSlicePoseFromSliceNormalAndPosition(redSliceNode, pelvis_origin_to_symph, pub_super_c)

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI:
 Once done checking, return the red slice to default poisiton (see panel above the slice, change 'reformat' to 'axial')</b>
   </div>  

## Ankle Landmarks: Tibia and Fibula

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, collect further key markers:</b>

1) _fibula_r_lat_malleol_tip_ and _fibula_l_lat_malleol_tip_ : these should be placed on inferiormost points of fibulas/lateral malleolae.

2) _tibia_r_med_malleol_tip_ and _tibia_l_med_malleol_tip_ : these should be placed on inferiormost points of tibias/medial malleolae.
   </div>  

## Knee Landmarks: Femora

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, provisionally place knee center points:</b>
    <em>knee_r_center</em> and <em>knee_l_center</em> : these should be placed at the top of the intercondilar groove on the posterior face of each distal femur.
   </div>

#### Knee Centers

In [18]:
# left femur -- orient planes to reflect provisional axes
# re-load lms node
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_r_center = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]

femur_l_y_axis = (femur_l_center-knee_l_center)/np.linalg.norm(femur_l_center-knee_l_center)
femur_r_y_axis = (femur_r_center-knee_r_center)/np.linalg.norm(femur_r_center-knee_r_center)
femur_l_y_axis

array([-4.82938871e-04, -2.86033085e-02,  9.99590725e-01])

#### Left Femur: medial and lateral condyles

In [21]:
# reorient red slice perpendicular to the femur_l_y_axis:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(
    redSliceNode, 
    femur_l_y_axis, 
    knee_l_center)



<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, place medial and lateral condyle points:</b>
<em>knee_l_lat</em> and <em>knee_l_med</em> : these should be placed at the most prominent points of the left knee. Scroll red slice up and down few steps to ensure the best positions.
   </div>

In [23]:
# recalculate and update the position of the knee center:
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_lat = lms_list[lms.GetControlPointIndexByLabel('knee_l_lat')]
knee_l_med = lms_list[lms.GetControlPointIndexByLabel('knee_l_med')]

knee_l_center = np.mean((knee_l_lat, knee_l_med), axis = 0)
femur_l_y_axis = (femur_l_center-knee_l_center)/np.linalg.norm(femur_l_center-knee_l_center)
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('knee_l_center'), 
                               knee_l_center[0], knee_l_center[1], knee_l_center[2])

#### Right Femur: medial and lateral condyles

In [24]:
# reorient red slice perpendicular to the femur_r_y_axis:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(
    redSliceNode, 
    femur_r_y_axis, 
    knee_r_center)

<div class="alert alert-block alert-info">
<b>In 3dSclier GUI, provisionally place medial and lateral condyle points:</b>
<em>knee_r_lat</em> and <em>knee_r_med</em> : these should be placed at the most prominent points of the left knee. Scroll red slice up and down few steps to ensure the best positions.
   </div>

In [27]:
# recalculate and update the position of the knee center:
lms = slicer.util.getNode('lms')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_lat = lms_list[lms.GetControlPointIndexByLabel('knee_r_lat')]
knee_r_med = lms_list[lms.GetControlPointIndexByLabel('knee_r_med')]


knee_r_center = np.mean((knee_r_lat, knee_r_med), axis = 0)
femur_r_y_axis = (femur_r_center-knee_r_center)/np.linalg.norm(femur_r_center-knee_r_center)
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('knee_r_center'), 
                               knee_r_center[0], knee_r_center[1], knee_r_center[2])

In [32]:
# return slices into the default positions
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

# Second Set of Landmarks
## Warp the Template to the Space of the MRI

In [15]:
reference_for_warping_node = slicer.util.getNode('lms')
reference_for_warping_locations = slicer.util.arrayFromMarkupsControlPoints(reference_for_warping_node)
reference_for_warping_names = [reference_for_warping_node.GetNthControlPointLabel(i) for i in range(reference_for_warping_node.GetNumberOfControlPoints())]

In [16]:
template_for_warpintg_locations = template_markers_df.loc[reference_for_warping_names].to_numpy()

In [9]:
tps_spline = ThinPlateSpline(alpha = 0.002)
tps_spline.fit(template_for_warpintg_locations, reference_for_warping_locations)


In [19]:
# this sell introduces an order to landmarks which is different from the order in the OpenSim model
# it is done for easy access to relevant points as they come up in the protocol

order = pd.read_csv(os.path.join(f'../../', 'templates', 'point_order.csv'), index_col='name')

In [23]:
order.index

Index(['torso_origin_in_pelvis', 'ASIS_l', 'ASIS_r', 'PSIS_l', 'PSIS_r',
       'pub_infer_c', 'pub_super_c', 'ilium_l', 'ilium_r',
       'pelvis_origin_in_ground',
       ...
       'tibia_r_epiph_yellow_posteriormost', 'tibia_r_ankle_yellow_anter',
       'tibia_r_epiph_green_medial_lower_epiph', 'fibula_r_as',
       'tibia_r_midshaft_anter', 'tibia_r_midshaft_poster',
       'fibula_r_midshaft_anter', 'tibia_r_anter_diaph_25',
       'tibia_r_anter_diaph_75', 'fibula_r_anter_diaph_75'],
      dtype='object', name='name', length=107)

In [29]:
template_markers_reordered_df = template_markers_df.loc[order.index]

In [30]:
second_set = tps_spline.transform(template_markers_reordered_df.to_numpy())

In [31]:
lms_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
lms_cloud.SetName('orientation')
slicer.util.updateMarkupsControlPointsFromArray(lms_cloud, second_set)
for i, name in enumerate(template_markers_df.index):
    lms_cloud.SetNthFiducialLabel(i, name)

## Adjust the Position of the Warped Landmarks

<div class="alert alert-block alert-info">
<b>From now on all work is done in 'orientation.mrk.json', i.e. the warped set of landmarks.</b>
    
<p><b>Adjust location of projected landmarks, starting from the top:</b></p>

_sacroiliac_r_ , _sacroiliac_l_ -- inferior point on the joint between sacrum and ilium

_5th_sacr_v_  --  antero-inferior point on the 5th sacral vertebra

_AIIS_r_, _AIIS_l_ -- a tip of the anterio-inferior iliac spine

_gr_troch_as_l_ and _gr_troch_as_r_ -- greater trochanter antero-superior -- defined as the anterior-bump on the frontal surface of the greater trochanter -- many muscles are attached here.

_gr_troch_lat_l_ and _gr_troch_lat_r_ -- greater trochanter lateral -- defined as the lateralmost point on the green slice through greater trochanter, it should lie posteriorly to the gr_troch_as point and be at the insertion of muscle tendons into the lateral wall of the greater trochanter. This point is usually locatied at the lower level of the GT.
    
 _gr_troch_ps_l_ and _gr_troch_ps_r_ -- greater trochanter postero-superior -- defined as the top posterior point on the superior surface of the greater trochanter.

 _ls_troch_l_ and _ls_troch_r_ -- lesser trochanters on right and left, defined as the most prominent point on the respective lesser trochanter, locate it by manupulating red and green views, drag and drop points to the correct location. 

<p></p>
    
*isch_spine_l* and *isch_spine_r* -- ischial spine -- sharp spine at the postero-superior part of the ischial bone, right and left. 
    
*isch_tuber_l* and *isch_tuber_r* -- ischial tuberosity -- defined as the posteriormost landmark on ischial tuberosity, best viewed on the yellow slice, on red slice the are located at the posterior point of ischial tuberosity triangle and, on green slice they are marked as a black endpoint of the bone.

_isch_infer_l_ and _isch_infer_r_ -- ischial [bone] inferior -- defined as antero-inferiormost point of the imprint of the ischio-pubic arch on the red slice (control on the green and yellow). To locate them, scroll the red slice down until ischial bones just dissapear. Then start moving the slice upwards unitl you see a narrow imprint of either right or left ischial bone. Place the corresponding right or left landmarks at the anterior end of the thin bone outline.


<p></p>

_ankle_l_med_ and _ankle_r_med_ -- medial malleolus on left and right tibias; defined as the most medially prominent point on tibial malleolus

_ankle_l_lat_ and _ankle_r_lat_
</div>

## Calculate and Update Further Points for Knees and Ankles

In [33]:
## note that knee needs two sets of points updated: the ones that will be in child and 
## the ones that will be in parent frames

lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_lat = lms_list[lms.GetControlPointIndexByLabel('knee_l_lat')]
knee_l_med = lms_list[lms.GetControlPointIndexByLabel('knee_l_med')]
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('knee_l_center_in_femur_l'), 
                               knee_l_center[0], knee_l_center[1], knee_l_center[2])

knee_r_lat = lms_list[lms.GetControlPointIndexByLabel('knee_r_lat')]
knee_r_med = lms_list[lms.GetControlPointIndexByLabel('knee_r_med')]
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('knee_r_center_in_femur_r'), 
                               knee_r_center[0], knee_r_center[1], knee_r_center[2])

fibula_l_lat_malleol_tip = lms_list[lms.GetControlPointIndexByLabel('fibula_l_lat_malleol_tip')]
tibia_l_med_malleol_tip = lms_list[lms.GetControlPointIndexByLabel('tibia_l_med_malleol_tip')]
talus_l_center_in_tibia = np.mean((fibula_l_lat_malleol_tip, tibia_l_med_malleol_tip), axis = 0)
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('talus_l_center_in_tibia'), 
                               talus_l_center_in_tibia[0], 
                               talus_l_center_in_tibia[1], 
                               talus_l_center_in_tibia[2])

fibula_r_lat_malleol_tip = lms_list[lms.GetControlPointIndexByLabel('fibula_r_lat_malleol_tip')]
tibia_r_med_malleol_tip = lms_list[lms.GetControlPointIndexByLabel('tibia_r_med_malleol_tip')]
talus_r_center_in_tibia = np.mean((fibula_r_lat_malleol_tip, tibia_r_med_malleol_tip), axis = 0)
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('talus_r_center_in_tibia'), 
                               talus_r_center_in_tibia[0], 
                               talus_r_center_in_tibia[1], 
                               talus_r_center_in_tibia[2])

ankle_l_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_l_lat')]
ankle_l_med = lms_list[lms.GetControlPointIndexByLabel('ankle_l_med')]
ankle_l_center = np.mean((ankle_l_lat, ankle_l_med), axis = 0)
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('ankle_l_center'), 
                               ankle_l_center[0], 
                               ankle_l_center[1], 
                               ankle_l_center[2])

ankle_r_lat = lms_list[lms.GetControlPointIndexByLabel('ankle_r_lat')]
ankle_r_med = lms_list[lms.GetControlPointIndexByLabel('ankle_r_med')]
ankle_r_center = np.mean((ankle_r_lat, ankle_r_med), axis = 0) 
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('ankle_r_center'), 
                               ankle_r_center[0], 
                               ankle_r_center[1], 
                               ankle_r_center[2])

## Femora

### Left Femur

#### Left Femur Diaphysis

In [34]:
## Left femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_mean = np.mean((knee_l_center, femur_l_center), axis = 0)

##### midshaft

In [35]:
femur_l_midshaft_anter = lms.GetControlPointIndexByLabel('femur_l_midshaft_anter')
lms.SetNthControlPointPosition(femur_l_midshaft_anter, 
                               lms_list[femur_l_midshaft_anter][0], 
                               lms_list[femur_l_midshaft_anter][1], 
                               femur_l_mean[2])

femur_l_midshaft_poster = lms.GetControlPointIndexByLabel('femur_l_midshaft_poster')
lms.SetNthControlPointPosition(femur_l_midshaft_poster, 
                               lms_list[femur_l_midshaft_poster][0], 
                               lms_list[femur_l_midshaft_poster][1], 
                               femur_l_mean[2])

In [36]:
## re-import all markers and place the red plane at the level of the femur_l_mean
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]

femur_l_mean = np.mean((knee_l_center, femur_l_center), axis = 0)

### reorient the red slice:
femur_l_y_axis = (femur_l_center - knee_l_center)/np.linalg.norm(femur_l_center - knee_l_center)
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_y_axis, femur_l_mean)

### shift green and yellow slices to the same point
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_l_mean)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_l_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the left femur diaphysis midshaft landmarks (<em>femur_l_midshaft_anter</em>, <em>femur_l_midshaft_poster</em>) position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

##### 25%

In [37]:
## re-import all markers and place the red plane at the level of the femur_l_mean
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
femur_l_mean = np.mean((knee_l_center, femur_l_center), axis = 0)
femur_l_y_axis = (femur_l_center - knee_l_center)/np.linalg.norm(femur_l_center - knee_l_center)

# place slices at the first quater and adjust the position of femur shaft points
femur_l_first_quater = np.mean((femur_l_mean, femur_l_center), axis = 0)

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_y_axis, femur_l_first_quater)
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_l_first_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_l_first_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the left femur diaphysis first quater shaft landmarks (<em>femur_l_poster_diaph_25</em>, <em>femur_l_anter_diaph_25</em>) position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

##### 75%

In [38]:
## re-import all markers and place the red plane at the level of the femur_l_mean
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
femur_l_mean = np.mean((knee_l_center, femur_l_center), axis = 0)
femur_l_y_axis = (femur_l_center - knee_l_center)/np.linalg.norm(femur_l_center - knee_l_center)

# place slices at the last quater and adjust the position of femur shaft points
femur_l_last_quater = np.mean((femur_l_mean, knee_l_center), axis = 0)

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_y_axis, femur_l_last_quater)
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_l_last_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_l_last_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the left femur diaphysis last quater shaft landmarks (<em>femur_l_poster_diaph_75</em>, <em>femur_l_anter_diaph_75</em>) position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

<!-- <div class="alert alert-block alert-info">
    <b>In GUI</b> <em>femur_shaft_center_l</em>  - center point in the middle of the femur shaft outline at the level of ls_troch_l.
    </div> -->

#### Left Femur Patella

In [41]:
## re-import all markers and place all slices at the knee_l_center
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_l_center = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]

### reorient the red slice:
femur_l_y_axis = (femur_l_center - knee_l_center)/np.linalg.norm(femur_l_center - knee_l_center)
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_l_y_axis, knee_l_center)

### place green and yellow slices at the same point
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_l_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_l_center)

<div class="alert alert-block alert-info">
    <b>Contunue using the same orientation of the red slice</b>, move the green slice to appreciate the shape of the patella. Adjust the lateral and medial points so that they mark the widest mediolateral axis of the patella. Then use the coordinate system rotation tool to rotate teh red slice so that both, medial and lateral points are on the same red slice. Place the anterior landmark on the frontal surface of the patella between the medial and lateral points. Place the superior point ('patella_l_anter') on the yellow slice at the posterior tip of the patella. Place the inferior point ('patella_l' and 'patella_l_in_femur') at the lowest tip of the bone.
    </div>

In [55]:
# return slices into the default positions on the right side
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

### Right Femur

#### Right Femur Diaphysis

##### midshaft

In [56]:
## Right femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
femur_r_center = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_mean = np.mean((knee_r_center, femur_r_center), axis = 0)

In [57]:
# Place femur midshaft points exactly between two joints.
femur_r_midshaft_anter = lms.GetControlPointIndexByLabel('femur_r_midshaft_anter')
lms.SetNthControlPointPosition(femur_r_midshaft_anter, 
                               lms_list[femur_r_midshaft_anter][0], 
                               lms_list[femur_r_midshaft_anter][1], 
                               femur_r_mean[2])

femur_r_midshaft_poster = lms.GetControlPointIndexByLabel('femur_r_midshaft_poster')
lms.SetNthControlPointPosition(femur_r_midshaft_poster, 
                               lms_list[femur_r_midshaft_poster][0], 
                               lms_list[femur_r_midshaft_poster][1], 
                               femur_r_mean[2])

In [75]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

### reorient the red slice:
femur_r_y_axis = (femur_r_center - knee_r_center)/np.linalg.norm(femur_r_center - knee_r_center)
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_y_axis, femur_r_mean)

### place green and yellow slices at the same point
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_r_mean)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_r_mean)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the right femur diaphysis midshaft landmarks (<em>femur_r_midshaft_anter</em>, <em>femur_r_midshaft_poster</em>) position to fit on the red slice. position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

##### 25%

In [59]:
## Re-load Right femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
femur_r_center = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_mean = np.mean((knee_r_center, femur_r_center), axis = 0)
femur_r_y_axis = (femur_r_center - knee_r_center)/np.linalg.norm(femur_r_center - knee_r_center)

# place slices at the first quater and adjust the position of femur shaft points
femur_r_first_quater = np.mean((femur_r_mean, femur_r_center), axis = 0)


redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_y_axis, femur_r_first_quater)
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_r_first_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_r_first_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the left femur diaphysis first quater shaft landmarks (<em>femur_r_poster_diaph_25</em>, <em>femur_r_anter_diaph_25</em>) position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

##### 75%

In [60]:
## Re-load Right femur
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
femur_r_center = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_mean = np.mean((knee_r_center, femur_r_center), axis = 0)
femur_r_y_axis = (femur_r_center - knee_r_center)/np.linalg.norm(femur_r_center - knee_r_center)

# place slices at the last quater and adjust the position of femur shaft points
femur_r_last_quater = np.mean((femur_r_mean, knee_r_center), axis = 0)

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_y_axis, femur_r_last_quater)
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), femur_r_last_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), femur_r_last_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust the left femur diaphysis first quater shaft landmarks (<em>femur_r_poster_diaph_75</em>, <em>femur_r_poster_diaph_75</em>) position to fit on the red slice. To do so, move green and yellow slices but keep the red one in place. The points should be located across the largest diameter of the femur midshaft with the posterior landmark at the apex of the teardrop shape.
</div>

#### Right Femur Patella

In [76]:
## re-import all markers locate planes at the knee_r_center
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

femur_r_center = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]

### reorient the red slice:
femur_r_y_axis = (femur_r_center - knee_r_center)/np.linalg.norm(femur_r_center - knee_r_center)
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, femur_r_y_axis, knee_r_center)

### place green and yellow slices at the same point
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

<div class="alert alert-block alert-info">
    <b>Contunue using the same orientation of the red slice</b>, move the green slice to appreciate the shape of the patella. Adjust the lateral and medial points so that they mark the widest mediolateral axis of the patella. Then use the coordinate system rotation tool to rotate teh red slice so that both, medial and lateral points are on the same red slice. Place the anterior landmark on the frontal surface of the patella between the medial and lateral points. Place the superior point ('patella_l_anter') on the yellow slice at the posterior tip of the patella. Place the inferior point ('patella_l' and 'patella_l_in_femur') at the lowest tip of the bone.
    </div>

In [83]:
# return slices into the default positions and center on the left knee:
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_l_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_l_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_l_center)

## Left Tibia 

#### Left epiphysis

In [8]:
# First, calculate tibial axes
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
talus_l_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_l_center_in_tibia')]

# calculate vertical axis
tibia_l_normal = (knee_l_center-talus_l_center_in_tibia)/np.linalg.norm(knee_l_center-talus_l_center_in_tibia)

In [9]:
# Second, reorient the red slice perpendicular to the vertical axis for the left tibia
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, knee_l_center)

# And center green and yellow slices in the knee center.
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_l_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_l_center)

In [10]:
femur_head_left = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
femur_head_left.SetName('tibia_l_outline')

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, use a new markup point list node and call it 'tibia_l_outline'. Move the red slice and place 20 - 30 points (more points will produce a better result) around the outline of the tibial plateau on the red slice at the level where it just crosses through all of the tibial plateau.
</div>

In [11]:
# import the result:
tibia_l_outline = slicer.util.getNode('tibia_l_outline')
outline_list = slicer.util.arrayFromMarkupsControlPoints(tibia_l_outline )

# calculate outline center 
tibia_l_outline_center = np.mean(outline_list, axis = 0)

# add the center to the outline node
#slicer.modules.markups.logic().AddControlPoint(tibia_l_outline_center[0], tibia_l_outline_center[1], tibia_l_outline_center[2])

In [12]:
# update tibia_l_center
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('tibia_l_center'), 
                               tibia_l_outline_center[0], tibia_l_outline_center[1], tibia_l_outline_center[2])

In [13]:
# reimport outline points
tibia_l_outline = slicer.util.getNode('tibia_l_outline')
outline_list = slicer.util.arrayFromMarkupsControlPoints(tibia_l_outline )

# subtract the center from the outline +
outline_centered = outline_list - tibia_l_outline_center

# calculate eigenvectors
val, vect = np.linalg.eig(np.matmul(outline_centered.T, outline_centered))

In [14]:
#rotate yellow slice
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, vect[0], tibia_l_outline_center)

In [16]:
# rotate green slice (try vect[1] then vect[2] and choose one that rotates the green slice along)
# the longest axis of the outline
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, vect[2], tibia_l_outline_center)

In [17]:
# Locate the red slice in tibia_l_outline_center
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_outline_center)

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust positions of the lateral and medial tibia points on the red slice so that they fall along the line of the green slice throught the centre. Then recalculate orientation planes of the tibia:
</div>

In [63]:
# reimport points
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_l_med = lms_list[lms.GetControlPointIndexByLabel('tibia_l_med')]
tibia_l_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_l_lat')]
talus_l_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_l_center_in_tibia')]

# calculate vertical axis
tibia_l_normal = (tibia_l_outline_center-talus_l_center_in_tibia)/np.linalg.norm(tibia_l_outline_center-talus_l_center_in_tibia)
#tibia_l_normal = (knee_l_center-talus_l_center_in_tibia)/np.linalg.norm(knee_l_center-talus_l_center_in_tibia)

In [64]:
# calculate postero-anterior axis
tibia_l_pa = plane_normal(tibia_l_med, talus_l_center_in_tibia, tibia_l_lat)

# calculate left-to-right axis
tibia_l_lr = (np.cross(tibia_l_pa, tibia_l_normal))/np.linalg.norm(np.cross(tibia_l_pa, tibia_l_normal))

In [65]:
# rotate green slice
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
#setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_l_pa, knee_l_center) #tibia_l_outline_center
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_l_pa, tibia_l_outline_center) #

# rotate yellow slice
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
#setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_l_lr, knee_l_center) #tibia_l_outline_center
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_l_lr, tibia_l_outline_center) #

<div class="alert alert-block alert-info">
 <b>Click tibia_l_center to align slices with the center of the left tibia.</b> Keep green and yellow slices in the same position and move the red slice to locate following points. 
    
<p>on the yellow slice:  
    <li> <em>tibia_l_epiph_yellow_posteriormost</em> </li> 
    <li> <em>tibia_l_epiph_yellow_anterior_plate</em> </li> 
    <li> <em>tibia_l_epiph_yellow_patel_tend_point</em> </li> 
    <li> <em>tibia_l_ankle_yellow_anter</em> </li> 
</p> 
<p>on the green slice: 
    <li> <em>tibia_l_epiph_green_medial_lower_epiph</em> - this point is defined as the lower corner of the tibial medial ephiphysis.</li>
</p>

</div>

<div class="alert alert-block alert-info">
<b>Now that red, green and yellow slices are all aligned with the tibia axes</b>, adjust the position of landmarks on fibula: <em>fibula_l_as</em> is the anterior tip of the proximal end of the bone; These are best seen on the yellow slice but can be also controlled on the red slice.
</div>

#### Left diaphysis

##### 50%

In [66]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

## Left tibia
fibula_l_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_l_midshaft_anter')
tibia_l_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_l_midshaft_anter')
tibia_l_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_l_midshaft_poster')

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
# ankle_l_center = lms_list[lms.GetControlPointIndexByLabel('ankle_l_center')]
talus_l_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_l_center_in_tibia')]

tibia_l_mean = np.mean((knee_l_center, talus_l_center_in_tibia), axis=0)
tibia_l_normal = (knee_l_center-talus_l_center_in_tibia)/np.linalg.norm(knee_l_center-talus_l_center_in_tibia)

In [67]:
### the next step will locate midshaft landmarks APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_l_midshaft_anter, 
                               lms_list[fibula_l_midshaft_anter][0], 
                               lms_list[fibula_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_anter, 
                               lms_list[tibia_l_midshaft_anter][0], 
                               lms_list[tibia_l_midshaft_anter][1], 
                               tibia_l_mean[2])
lms.SetNthControlPointPosition(tibia_l_midshaft_poster, 
                               lms_list[tibia_l_midshaft_poster][0], 
                               lms_list[tibia_l_midshaft_poster][1], 
                               tibia_l_mean[2])

In [68]:
### reorient the red slice and center it on the centre of the left tibia:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_mean)

## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), lms_list[tibia_l_midshaft_anter])
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), lms_list[tibia_l_midshaft_poster])

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, keep the red slice in place, move green and yellow slices to find and adjust position of midshaft landmarks (**tibia_l_midshaft_anter**, **tibia_l_midshaft_poster**, **fibula_l_midshaft_anter**) to fit on the red slice exactly anterior or posterior of the shaft (across its longest axis). The anterior landmark should be placed at the tip of the sharp corner anteriorly.
</div>

##### 25%

In [69]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
talus_l_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_l_center_in_tibia')]
tibia_l_mean = np.mean((knee_l_center, talus_l_center_in_tibia), axis=0)
tibia_l_normal = (knee_l_center - talus_l_center_in_tibia)/np.linalg.norm(knee_l_center - talus_l_center_in_tibia)

tibia_l_top_quater = np.mean((knee_l_center,tibia_l_mean), axis = 0)

### reorient the red slice and center it on the centre of the left tibia:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_top_quater)
## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), tibia_l_top_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), tibia_l_top_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, on the red slice, place <b><em> tibia_l_anter_diaph_25 </em></b> point -- which is at the anterior sharp corner of tibia (place it just at the edge/outside of the black cortical bone) .
</div>

##### 75%

In [70]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_l_center = lms_list[lms.GetControlPointIndexByLabel('knee_l_center')]
talus_l_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_l_center_in_tibia')]
tibia_l_mean = np.mean((knee_l_center, talus_l_center_in_tibia), axis=0)
tibia_l_normal = (knee_l_center-talus_l_center_in_tibia)/np.linalg.norm(knee_l_center-talus_l_center_in_tibia)

tibia_l_bottom_quater = np.mean((talus_l_center_in_tibia, tibia_l_mean), axis = 0)

### reorient the red slice and center it on the centre of the left tibia:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_l_normal, tibia_l_bottom_quater)
## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), tibia_l_bottom_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), tibia_l_bottom_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, on the red slice, place <b><em> tibia_l_anter_diaph_75 </em></b> and <b><em> fibula_l_anter_diaph_75 </em></b> points -- which is at the anterior sharp corner of tibia (place it just at the edge/outside of the black cortical bone) .
</div>

In [26]:
# return slices into the default positions and center on the right knee:
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)
knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]

redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

## Right Tibia

#### Right epiphysis

In [27]:
# First, calculate tibial axes
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
talus_r_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_r_center_in_tibia')]

# calculate vertical axis
tibia_r_normal = (knee_r_center-talus_r_center_in_tibia)/np.linalg.norm(knee_r_center-talus_r_center_in_tibia)

In [28]:
# Then, reorient red slice perpendicular to ankle-knee axis for the right tibia
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, knee_r_center)

# And center green and yellow slices in the knee center.
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

In [29]:
femur_head_left = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
femur_head_left.SetName('tibia_r_outline')

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, use the new markup point list node and call it 'tibia_r_outline'. Move the red slice and place 20-30 (more points will produce better result) points arounf the outline of the tibial plateau on the red slice at the level were it just crosses through all of the tibial plateau.
</div>

In [42]:
# import the result
tibia_r_outline = slicer.util.getNode('tibia_r_outline')
outline_list = slicer.util.arrayFromMarkupsControlPoints(tibia_r_outline )

# calculate the center 
tibia_r_outline_center = np.mean(outline_list, axis = 0)

In [43]:
# update tibia_r_center
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('tibia_r_center'), 
                               tibia_r_outline_center[0], tibia_r_outline_center[1], tibia_r_outline_center[2])

In [45]:
# in the next step, we generate eigenvectors of the tibial plateau outline
# and chose those that correspond with the mediolateral and anteroposterior directions

# reimport the outline
tibia_r_outline = slicer.util.getNode('tibia_r_outline')
outline_list = slicer.util.arrayFromMarkupsControlPoints(tibia_r_outline )

# subtract the center from the outline +
outline_centered = outline_list - tibia_r_outline_center

# calculate eigenvectors
val, vect = np.linalg.eig(np.matmul(outline_centered.T, outline_centered))

In [46]:
# rotate yellow slice
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, vect[0], tibia_r_outline_center) #tibia_r_outline_center

In [49]:
# rotate green slice (try vect[1] then vect[2] and choose one that rotates the green slice along)
# the longest axis of the outline
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, vect[1], tibia_r_outline_center) #tibia_r_outline_center

In [48]:
# Locate the red slice in tibia_l_outline_center
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_outline_center) #tibia_r_outline_center

<div class="alert alert-block alert-info">
<b>In 3dSlicer</b>, adjust positions of the lateral and medial tibia points to fit on the red slice along the line of the green slice throught the centre.
</div>

In [71]:
# import points
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

tibia_r_med = lms_list[lms.GetControlPointIndexByLabel('tibia_r_med')]
tibia_r_lat = lms_list[lms.GetControlPointIndexByLabel('tibia_r_lat')]
talus_r_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_r_center_in_tibia')]

In [72]:
# calculate postero-anterior axis
tibia_r_pa = plane_normal(tibia_r_med, talus_r_center_in_tibia, tibia_r_lat)

# calculate left-to-right axis
tibia_r_lr = (np.cross(tibia_r_pa, tibia_r_normal))/np.linalg.norm(np.cross(tibia_r_pa, tibia_r_normal))

In [73]:
# rotate green slice
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_r_pa, tibia_r_outline_center) #
#setSlicePoseFromSliceNormalAndPosition(greenSliceNode, tibia_r_pa, knee_r_center) #tibia_r_outline_center

# rotate yellow slice
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_r_lr, tibia_r_outline_center) #
#setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, tibia_r_lr, knee_r_center) #tibia_r_outline_center

<div class="alert alert-block alert-info">
 <b>Click tibia_r_center to align slices with the center of the left tibia.</b> Keep green and yellow slices in the same position and move the red slice to locate following points. 
    <p>on the yellow slice:  
    <li> <em>tibia_r_epiph_yellow_posteriormost</em> - defined as the posterior inferior point of the proximal epiphysis.</li> 
    <li> <em>tibia_r_epiph_yellow_anterior_plate</em> </li> 
    <li> <em>tibia_r_epiph_yellow_patel_tend_point</em> </li> 
    <li> <em>tibia_r_ankle_yellow_anter</em> </li> 
    </p> 
    <p>on the green slice: 
    <li> <em>tibia_r_epiph_green_medial_lower_epiph</em> - this point is defined as the lower corner of the tibial medial ephiphysis. </li></p>

</div>

<div class="alert alert-block alert-info">
<b>Contunue using the same orientation of slices</b>, adjust the position of landmarks on fibula: <em>fibula_r_as</em> is the anterior tip of the proximal end of the bone, best seen on the yellow slice but can be also controlled on the red slice.
</div>

In [53]:
# return slices into the default positions
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

#### Right diaphysis

##### 50%

In [54]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

## Right tibia
fibula_r_midshaft_anter = lms.GetControlPointIndexByLabel('fibula_r_midshaft_anter')
tibia_r_midshaft_anter = lms.GetControlPointIndexByLabel('tibia_r_midshaft_anter')
tibia_r_midshaft_poster = lms.GetControlPointIndexByLabel('tibia_r_midshaft_poster')

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
ankle_r_center = lms_list[lms.GetControlPointIndexByLabel('ankle_l_center')]
talus_r_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_r_center_in_tibia')]

tibia_r_mean = np.mean((knee_r_center,ankle_r_center), axis=0)
tibia_r_normal = (knee_r_center-talus_r_center_in_tibia)/np.linalg.norm(knee_r_center-talus_r_center_in_tibia)

In [55]:
### the next step will locate midshaft landmars APPROXIMATELY in the middle of diaphisis
lms.SetNthControlPointPosition(fibula_r_midshaft_anter, 
                               lms_list[fibula_r_midshaft_anter][0], 
                               lms_list[fibula_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_anter, 
                               lms_list[tibia_r_midshaft_anter][0], 
                               lms_list[tibia_r_midshaft_anter][1], 
                               tibia_r_mean[2])
lms.SetNthControlPointPosition(tibia_r_midshaft_poster, 
                               lms_list[tibia_r_midshaft_poster][0], 
                               lms_list[tibia_r_midshaft_poster][1], 
                               tibia_r_mean[2])

In [56]:
### reorient the red slice:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_mean)

## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), lms_list[tibia_r_midshaft_anter])
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), lms_list[tibia_r_midshaft_poster])

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, keep the red slice in place, move green and yellow slices to find and adjust position of midshaft landmarks (<em>tibia_r_midshaft_anter</em>, <em>tibia_r_midshaft_poster</em>, <em>fibula_r_midshaft_anter</em>)to fit on the red slice exactly anterior or posterior of the shaft (across its longest axis). The anterior landmark should be placed at the tip of the sharp corner anteriorly.
</div>

##### 25%

In [60]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
talus_r_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_r_center_in_tibia')]
tibia_r_mean = np.mean((knee_r_center, talus_r_center_in_tibia), axis=0)
tibia_r_normal = (knee_r_center - talus_r_center_in_tibia)/np.linalg.norm(knee_r_center - talus_r_center_in_tibia)

tibia_r_top_quater = np.mean((knee_r_center, tibia_r_mean), axis = 0)

### reorient the red slice and center it on the centre of the left tibia:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_top_quater)
## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), tibia_r_top_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), tibia_r_top_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, on the red slice, place <b><em> tibia_r_anter_diaph_25 </em></b> point -- which is at the anterior sharp corner of tibia (place it just at the edge/outside of the black cortical bone) .
</div>

##### 75%

In [61]:
## re-import all markers
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

knee_r_center = lms_list[lms.GetControlPointIndexByLabel('knee_r_center')]
talus_r_center_in_tibia = lms_list[lms.GetControlPointIndexByLabel('talus_r_center_in_tibia')]
tibia_r_mean = np.mean((knee_r_center, talus_r_center_in_tibia), axis=0)
tibia_r_normal = (knee_r_center-talus_r_center_in_tibia)/np.linalg.norm(knee_r_center-talus_r_center_in_tibia)

tibia_r_bottom_quater = np.mean((talus_r_center_in_tibia, tibia_r_mean), axis = 0)

### reorient the red slice and center it on the centre of the left tibia:
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, tibia_r_normal, tibia_r_bottom_quater)
## move yellow and green slices to show midshaft points
yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), tibia_r_bottom_quater)
greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), tibia_r_bottom_quater)

<div class="alert alert-block alert-info">
<b>In 3dSlicer GUI</b>, on the red slice, place <b><em> tibia_r_anter_diaph_75 </em></b> and <b><em> fibula_r_anter_diaph_75 </em></b> points -- which is at the anterior sharp corner of tibia (place it just at the edge/outside of the black cortical bone) .
</div>

In [74]:
# return slices into the default positions
redSliceNode = slicer.util.getNode("vtkMRMLSliceNodeRed")
setSlicePoseFromSliceNormalAndPosition(redSliceNode, np.array([0,0,1]), knee_r_center)

yellowSliceNode = slicer.util.getNode("vtkMRMLSliceNodeYellow")
setSlicePoseFromSliceNormalAndPosition(yellowSliceNode, np.array([1,0,0]), knee_r_center)

greenSliceNode = slicer.util.getNode("vtkMRMLSliceNodeGreen")
setSlicePoseFromSliceNormalAndPosition(greenSliceNode, np.array([0,1,0]), knee_r_center)

# Finishing points

In [375]:
# Update location of joints in parent frames that were previousely omitted -- these should have been projected into the correct location anyway but just to make sure
lms = slicer.util.getNode('orientation')
lms_list = slicer.util.arrayFromMarkupsControlPoints(lms)

pelvis_origin_in_ground_index = lms.GetControlPointIndexByLabel('pelvis_origin_in_ground')
pelvis_origin_coords = lms_list[lms.GetControlPointIndexByLabel('pelvis_origin')]
lms.SetNthControlPointPosition(pelvis_origin_in_ground_index, 
                               pelvis_origin_coords[0], 
                               pelvis_origin_coords[1], 
                               pelvis_origin_coords[2])

femur_l_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_l_center')]
femur_l_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_l_center_in_pelvis')
lms.SetNthControlPointPosition(femur_l_center_in_pelvis_index, 
                               femur_l_center_coords[0], 
                               femur_l_center_coords[1], 
                               femur_l_center_coords[2])

femur_r_center_coords = lms_list[lms.GetControlPointIndexByLabel('femur_r_center')]
femur_r_center_in_pelvis_index = lms.GetControlPointIndexByLabel('femur_r_center_in_pelvis')
lms.SetNthControlPointPosition(femur_r_center_in_pelvis_index, 
                               femur_r_center_coords[0], 
                               femur_r_center_coords[1], 
                               femur_r_center_coords[2])

patella_l = lms_list[lms.GetControlPointIndexByLabel('patella_l')]
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('patella_l_in_femur_l'), 
                               patella_l[0], 
                               patella_l[1], 
                               patella_l[2])

patella_r = lms_list[lms.GetControlPointIndexByLabel('patella_r')]
lms.SetNthControlPointPosition(lms.GetControlPointIndexByLabel('patella_r_in_femur_r'), 
                               patella_r[0], 
                               patella_r[1], 
                               patella_r[2])

# Dont forget to save your work!